<a href="https://colab.research.google.com/github/AlexandreMinato/Investimentos_FII/blob/main/Analises_FII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [182]:
import datetime
import logging
import requests
import numpy as np
import pandas as pd
import pytz

# Capturando os dados

In [184]:
url = 'https://www.fundsexplorer.com.br/ranking'
dados = requests.get(url,  headers={'User-Agent': 'Mozilla/5.0'})
list_obj = pd.read_html(dados.text, attrs={'id': 'table-ranking'})
df = list_obj[0]

In [185]:
# redefinindo nome das colunas
df.columns = ['ativo', 'setor', 'preco', 'liquidez_dia', 'dividendo', 'yeld', 'yeld_3m_acululado', 'yeld_6m_acululado', 'yeld_12m_acululado', 'yeld_3m_media', 'yeld_6m_media', 'yeld_12m_media', 'yeld_ano', 'variacao_preco', 'rent_periodo', 'rent_acumulada', 'patrimonio_liq', 'vpa', 'p_vpa', 'dy_patrimonial', 'variacao_patrimonial', 'rent_patr_periodo', 'rent_acumulada_periodo', 'vacancia_fisica', 'vacancia_financeira', 'quantidade_ativos']

In [126]:
df.head()

,ativo,setor,preco,liquidez_dia,dividendo,yeld,yeld_3m_acululado,yeld_6m_acululado,yeld_12m_acululado,yeld_3m_media,yeld_6m_media,yeld_12m_media,yeld_ano,variacao_preco,rent_periodo,rent_acumulada,patrimonio_liq,vpa,p_vpa,dy_patrimonial,variacao_patrimonial,rent_patr_periodo,rent_acumulada_periodo,vacancia_fisica,vacancia_financeira,quantidade_ativos
0,FIVN11,Shoppings,"R$ 3,09",31730.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%",NaN,"0,00%","0,00%","0,00%","R$ 64.832.865,32","R$ 6,89",45.0,NaN,NaN,NaN,NaN,"56,00%",NaN,1
1,BZLI11,Títulos e Val. Mob.,"R$ 14,70",44041.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,68%","0,68%","1,14%","R$ 439.665.897,40","R$ 11,45",128.0,NaN,NaN,NaN,NaN,NaN,NaN,0
2,XTED11,Lajes Corporativas,"R$ 6,88",993.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","1,10%","1,10%","-37,83%","R$ 25.472.636,57","R$ 14,21",48.0,NaN,NaN,NaN,NaN,"0,00%","100,00%",1
3,ALMI11,Lajes Corporativas,"R$ 1.180,00",31.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","2,27%","2,27%","-28,29%","R$ 229.423.903,28","R$ 2.063,59",57.0,NaN,NaN,NaN,NaN,"68,15%",NaN,1
4,DOMC11,Lajes Corporativas,"R$ 439,00",42.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","4,35%","4,35%","-10,00%","R$ 264.351.778,61","R$ 108,17",406.0,NaN,NaN,NaN,NaN,"23,44%",NaN,1


#Normalizando as colunas



In [127]:
def identificar_conteudo_colunas(df, item):
    
    return [df[column].name for column in df if df[column].astype(str).str.contains(item).any()]

In [128]:
#Converter financeiro
coluna_monetario = identificar_conteudo_colunas(df, '\$')

for column_name in coluna_monetario:
    df[column_name] = df[column_name].str.replace('.', '')
    df[column_name] = df[column_name].str.replace(',', '.')
    df[column_name] = df[column_name].str.replace('[R\$ ,]', '', regex=True).astype(float)

In [129]:
# Converter as colunas de percentual em numericas
coluna_percentual = identificar_conteudo_colunas(df, '\%')
for column_name in coluna_percentual:
    df[column_name] = df[column_name].str.replace(',', '.')
    df[column_name] = df[column_name].str.replace('[%,]', '', regex=True).astype(float)
    df[column_name] = df[column_name]/100

In [130]:
# Corrige o valor patrimonial
df['p_vpa'] = df['p_vpa']/100

In [131]:
# Normaliza o setor
df['setor'] = df['setor'].str.replace('\s+', '_')
df['setor'] = df['setor'].str.replace(r'[^\w\s]+', '_')
df['setor'] = df['setor'].str.lower()
df['setor'] = df['setor'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# Iniciando as analises


In [132]:
df.setor.unique()

array(['shoppings', 'titulos_e_val__mob_', 'lajes_corporativas',
       'hibrido', 'outros', 'hotel', 'residencial', 'logistica',
       'hospital'], dtype=object)

In [133]:
df.ativo.nunique()

238

In [134]:
# Convert p/vpa to the correct base
df['p_vpa'] = df['p_vpa']/100
# Checking if is it correct
df[df['ativo'] == 'VRTA11']

,ativo,setor,preco,liquidez_dia,dividendo,yeld,yeld_3m_acululado,yeld_6m_acululado,yeld_12m_acululado,yeld_3m_media,yeld_6m_media,yeld_12m_media,yeld_ano,variacao_preco,rent_periodo,rent_acumulada,patrimonio_liq,vpa,p_vpa,dy_patrimonial,variacao_patrimonial,rent_patr_periodo,rent_acumulada_periodo,vacancia_fisica,vacancia_financeira,quantidade_ativos
106,VRTA11,titulos_e_val__mob_,112.38,17785.0,0.9,0.0078,0.0235,0.0461,0.0839,0.0078,0.0077,0.007,0.0313,0.006,0.0138,0.0304,1.010653e+09,101.07,0.0111,NaN,NaN,NaN,NaN,NaN,NaN,0


In [135]:
df.describe()

,preco,liquidez_dia,dividendo,yeld,yeld_3m_acululado,yeld_6m_acululado,yeld_12m_acululado,yeld_3m_media,yeld_6m_media,yeld_12m_media,yeld_ano,variacao_preco,rent_periodo,rent_acumulada,patrimonio_liq,vpa,p_vpa,dy_patrimonial,variacao_patrimonial,rent_patr_periodo,rent_acumulada_periodo,vacancia_fisica,vacancia_financeira,quantidade_ativos
count,228.000000,2.280000e+02,238.000000,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,212.000000,222.000000,222.000000,222.000000,2.380000e+02,238.000000,228.0000,4.000000,4.000000,4.000000,4.000000,116.000000,25.000000,238.000000
mean,519.575088,2.267750e+04,1.316261,0.007403,0.018024,0.032010,0.051234,0.006009,0.005334,0.004271,0.025496,-0.014414,-0.007442,-0.031715,5.149004e+08,662.396429,inf,0.006950,0.002075,0.009000,0.022475,0.155532,0.126372,3.331933
std,5642.306143,1.006695e+05,4.445995,0.024198,0.026794,0.033587,0.051449,0.008929,0.005599,0.004289,0.028759,0.103475,0.103215,0.123874,7.104465e+08,6211.667336,NaN,0.001109,0.003292,0.003562,0.004882,0.228208,0.232114,7.999150
min,0.780000,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.946500,-0.946200,-0.945900,-5.574880e+06,-7.350000,-0.0172,0.005900,0.000100,0.006500,0.016200,0.000000,0.000000,0.000000
25%,82.282500,2.182500e+02,0.400000,0.004100,0.009200,0.015675,0.000000,0.003050,0.002625,0.000000,0.016075,-0.022675,-0.016175,-0.050900,1.030673e+08,93.750000,0.0084,0.006200,0.000475,0.006800,0.019875,0.000000,0.000000,0.000000
50%,99.120000,2.483000e+03,0.630000,0.006100,0.016700,0.032400,0.055600,0.005550,0.005400,0.004600,0.022450,-0.000500,0.000400,-0.009900,2.621177e+08,99.915000,0.0097,0.006750,0.000600,0.007650,0.023350,0.057350,0.036000,1.000000
75%,111.545000,1.481425e+04,0.900000,0.007675,0.022200,0.043675,0.077175,0.007400,0.007275,0.006400,0.030100,0.013275,0.018775,0.023900,5.922617e+08,109.795000,0.0104,0.007500,0.002200,0.009850,0.025950,0.211000,0.123000,3.000000
max,85227.760000,1.336487e+06,52.740000,0.363200,0.379900,0.409600,0.452200,0.126600,0.068300,0.037700,0.392400,0.327300,0.330300,0.410700,4.852707e+09,93717.970000,inf,0.008400,0.007000,0.014200,0.027000,1.000000,1.000000,72.000000


In [136]:
df.setor.unique()

array(['shoppings', 'titulos_e_val__mob_', 'lajes_corporativas',
       'hibrido', 'outros', 'hotel', 'residencial', 'logistica',
       'hospital'], dtype=object)

In [137]:
# Filtering relevant sectors
df = df[df['setor'].isin(['shoppings', 'titulos_e_val__mob_', 'lajes_corporativas', 'hibrido', 'logistica'])].copy()

In [138]:
df.setor.unique()

array(['shoppings', 'titulos_e_val__mob_', 'lajes_corporativas',
       'hibrido', 'logistica'], dtype=object)

In [139]:
df.head()

,ativo,setor,preco,liquidez_dia,dividendo,yeld,yeld_3m_acululado,yeld_6m_acululado,yeld_12m_acululado,yeld_3m_media,yeld_6m_media,yeld_12m_media,yeld_ano,variacao_preco,rent_periodo,rent_acumulada,patrimonio_liq,vpa,p_vpa,dy_patrimonial,variacao_patrimonial,rent_patr_periodo,rent_acumulada_periodo,vacancia_fisica,vacancia_financeira,quantidade_ativos
0,FIVN11,shoppings,3.09,31730.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0000,0.0000,0.0000,6.483287e+07,6.89,0.0045,NaN,NaN,NaN,NaN,0.5600,NaN,1
1,BZLI11,titulos_e_val__mob_,14.70,44041.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0068,0.0068,0.0114,4.396659e+08,11.45,0.0128,NaN,NaN,NaN,NaN,NaN,NaN,0
2,XTED11,lajes_corporativas,6.88,993.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0110,0.0110,-0.3783,2.547264e+07,14.21,0.0048,NaN,NaN,NaN,NaN,0.0000,1.0,1
3,ALMI11,lajes_corporativas,1180.00,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0227,0.0227,-0.2829,2.294239e+08,2063.59,0.0057,NaN,NaN,NaN,NaN,0.6815,NaN,1
4,DOMC11,lajes_corporativas,439.00,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0435,0.0435,-0.1000,2.643518e+08,108.17,0.0406,NaN,NaN,NaN,NaN,0.2344,NaN,1


In [149]:
# Price vs net worth
df['acceptable_pricing'] = np.where((df['p_vpa'] > 0.9) & (df['p_vpa'] < 1.1) , True, False)

# Asset diversification
diverssified_assets_conditions = [
    # 1st Condition
    (df['setor'].isin(['shoppings', 'lajes_corporativas', 'logistica'])) &
    (df['quantidade_ativos'] > 5),
    # 2st Condition
    (df['setor'].isin(['shoppings', 'lajes_corporativas', 'logistica'])) &
    (df['quantidade_ativos'] < 5),
    # 3rd Condition
    (~df['setor'].isin(['shoppings', 'lajes_corporativas', 'logistica']))
]
diverssified_assets_choices = [True, False, True]
df['diverssified_assets'] = np.select(diverssified_assets_conditions, diverssified_assets_choices, default=None)

# Vacancy
df['razonably_occupied'] = np.where((df['vacancia_fisica'] < 0.1) | (df['vacancia_fisica'].isnull()) | (df['vacancia_fisica'].isnull()) , True, False)

# Liquidity
df['is_liquid'] = np.where(df['liquidez_dia'] > 20000, True, False)

# dividend yield
df['good_earnings'] = np.where(df['yeld_12m_acululado'] > 0.05, True, False)

In [151]:
papper_acceptable_conditions = (
    df.acceptable_pricing &
    df.diverssified_assets &
    df.razonably_occupied &
    df.is_liquid &
    df.good_earnings
)
df_results = df.loc[papper_acceptable_conditions].sort_values(by=['yeld_12m_acululado', 'liquidez_dia'], ascending=False)

### Resultado
*Os papéis abaixo não são recomendação de compra, mas sim estudo*.

In [152]:
df_results

,ativo,setor,preco,liquidez_dia,dividendo,yeld,yeld_3m_acululado,yeld_6m_acululado,yeld_12m_acululado,yeld_3m_media,yeld_6m_media,yeld_12m_media,yeld_ano,variacao_preco,rent_periodo,rent_acumulada,patrimonio_liq,vpa,p_vpa,dy_patrimonial,variacao_patrimonial,rent_patr_periodo,rent_acumulada_periodo,vacancia_fisica,vacancia_financeira,quantidade_ativos,preco_justo,diversificacoes,ocupacao,liquidez,bons_dividendos,acceptable_pricing,diverssified_assets,razonably_occupied,is_liquid,good_earnings
